In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 14.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 36.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platf

In [2]:
from huggingface_hub import login

login()

In [3]:
import csv
import json
import logging
import os
from typing import Optional, Any, TypedDict

import numpy as np
import scipy.io.wavfile
from tqdm import tqdm

import torch
from torch.utils.data import Dataset
from transformers import pipeline, Pipeline

from concurrent.futures import ThreadPoolExecutor, as_completed

logger = logging.getLogger(__name__)

2025-10-24 17:08:07.614537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761325687.788151      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761325687.841128      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
class PromptItemLoader(TypedDict):
    id: str
    prompt: str


class OutputGeneratedAudioItem(TypedDict):
    id: str
    prompt: str
    audio_path: str

In [5]:
def read_json(file_path: str) -> dict[str, Any] | list[dict[str, Any]]:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def read_csv(file_path: str) -> dict[str, Any]:
    with open(file_path, "r", encoding="utf-8") as f:
        csv_reader = csv.DictReader(f)
        rows = [row for row in csv_reader]

    return rows


class DatasetLoader(Dataset):
    """
    Dataset para representar input data para el modelo `taste-music-dataset`.
    Attributes:
        file_path (str): La ruta al archivo JSON que contiene los datos de Spanio.
    """

    def __init__(self, file_path, format_: str = "json"):
        """
        son_file_path (str): Ruta del archivo input con prompts.
        """
        super().__init__()
        self.file_path = file_path
        self.format_ = format_
        self.records: list[PromptItemLoader] = []
        self._load_data(self.format_)

    def _load_data(self, file_format: str) -> None:
        read_functions = {
            "json": read_json,
            "csv": read_csv,
        }
        try:
            self.records = read_functions[file_format](self.file_path)
        except Exception as e:
            print(f"Error cargando {self.file_path}: {e}")
            logger.critical(e, exc_info=True)

    def __len__(self) -> int:
        """
        Returns:
            int: Número de registros disponibles.
        """
        return len(self.records)

    def __getitem__(self, idx) -> PromptItemLoader:
        """
        Parameters:
            idx (int): Índice del registro a retornar.
        Returns
            El item del dataset en la posicion recibida por parametro.
        """
        if not 0 <= idx < len(self.records):
            raise IndexError(
                f"indice {idx} fuera de rango para dataset: {len(self.records)}."
            )
        return self.records[idx]

In [6]:
def load_musicgen_pipeline(
    model_name: str = "csc-unipd/tasty-musicgen-small",
    prefer_gpu: bool = True,
    dtype: Optional[torch.dtype] = None,
) -> Pipeline:
    device = "cuda:0" if prefer_gpu and torch.cuda.is_available() else "cpu"
    if dtype is None and device.startswith("cuda"):
        dtype = torch.float16

    pipe: Pipeline = pipeline(
        task="text-to-audio", model=model_name, torch_dtype=dtype, device=device
    )

    return pipe

In [8]:
def generate_audio_from_prompts(
    synthesiser, dataset, output_dir="generated_music", sample_rate=32000
):
    os.makedirs(output_dir, exist_ok=True)
    results = []

    print(f"Generando música para {len(dataset)} prompts...\n")

    for record in tqdm(dataset.records):
        text_prompt = record["prompt"]
        file_id = record["id"]

        try:
            # 1. Generar la música con el modelo.
            # output es un diccionario: audio(array NumPy con la señal de audio) y sampling_rate (frecuencia de muestreo del modelo).
            output = synthesiser(text_prompt, forward_params={"do_sample": True})

            # 2. Extraer datos del audio.
            audio_data = output["audio"]  # La onda de sonido (las muestras del audio).
            sr = output.get(
                "sampling_rate", sample_rate
            )  # La frecuencia de muestreo reportada por el modelo.

            # 3. Guardar el audio generado.
            output_path = os.path.join(output_dir, f"{file_id}.wav")
            scipy.io.wavfile.write(
                output_path, rate=sr, data=audio_data
            )  # Escribir el archivo .wav con la señal y la frecuencia.

            # 4. Registrar los resultados.
            results.append(
                {"id": file_id, "description": text_prompt, "audio_path": output_path}
            )
        except Exception as e:
            print(f" Error generando {file_id}: {e}")
            continue

    print(f"\n {len(results)} archivos de audio generados en: {output_dir}")
    return results

In [ ]:
dataset = DatasetLoader("/kaggle/input/prompts/spanio_prompts_v2.csv", format_="csv")
synth = load_musicgen_pipeline()

generate_audio_from_prompts(
    synthesiser=synth, dataset=dataset, output_dir="/kaggle/working/generated_music"
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Generando música para 100 prompts...



  6%|▌         | 6/100 [03:45<58:30, 37.35s/it]  